<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/separatemodeling_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [24]:
import pandas as pd
import numpy as np

from ddop.newsvendor import KNeighborsWeightedNewsvendor
from ddop.newsvendor import SampleAverageApproximationNewsvendor

from sklearn.model_selection import RandomizedSearchCV

from ddop.metrics import make_scorer
from ddop.metrics import average_costs
from ddop.metrics import prescriptiveness_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading data-set

*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**



In [4]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

## Split in feature and target matrix

In [5]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split

In [6]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

## Definition of the item_ids 


*   will be needed within the for loops later
*   will contain all 25 products



In [7]:
prods = list(range(0,25))

cu = round(X_train['sell_price'].mean(),2)

co_90 = round(((cu-(0.90*cu))/0.90),2)

co_75 = round(((cu-(0.75*cu))/0.75),2)

co_50 = round(((cu-(0.50*cu))/0.50),2)

## Determining some parameters
*   the potential params and their values for randomized search are defined
*   the estimator, the scorer and finally the whole grid is defined





In [10]:
k = np.arange(25,66)
param_random = dict(n_neighbors = k)

scorer_avc = make_scorer(average_costs, greater_is_better=False)

# knn for 90% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,90
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_k_90 = []
for prod in prods:
  
  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_90)
  
  random = RandomizedSearchCV(knn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
  
  k = random.best_params_.get('n_neighbors')
  best_param_k_90.append(k)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [19]:
result_knn_avc_90 = []
for prod in prods:

  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_90, n_neighbors = best_param_k_90[prod])
  knn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = knn.predict(X_test.loc[prod])
  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_90),2)
  result_knn_avc_90.append(avc)

In [21]:
KNN_separate_avc_90 = pd.DataFrame(result_knn_avc_90, columns=['KNN_avc_90'])
KNN_separate_avc_90.index.name = 'item_id'

In [25]:
result_knn_90_pscr = []
for prod in prods:

  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_90, n_neighbors = best_param_k_90[prod])
  knn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_knn = knn.predict(X_test.loc[prod])

  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_90)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)



  pscr = prescriptiveness_score(Y_test.loc[prod], preds_knn, preds_saa ,cu = cu, co = co_90)
  result_knn_90_pscr.append(pscr)

In [27]:
KNN_separate_pscr_90 = pd.DataFrame(result_knn_90_pscr, columns=['KNN_pscr_90'])
KNN_separate_pscr_90.index.name = 'item_id'

# knn for 75% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,75
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_k_75 = []
for prod in prods:
  
  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_75)
  
  random = RandomizedSearchCV(knn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
  
  k =  k = random.best_params_.get('n_neighbors')
  best_param_k_75.append(k)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [33]:
result_knn_avc_75 = []
for prod in prods:
  
  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_75, n_neighbors = best_param_k_75[prod])
  knn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = knn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_75),2)
  result_knn_avc_75.append(avc)

In [34]:
KNN_separate_avc_75 = pd.DataFrame(result_knn_avc_75, columns=['KNN_avc_75'])
KNN_separate_avc_75.index.name = 'item_id'

In [36]:
result_knn_75_pscr = []
for prod in prods:
  
  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_75, n_neighbors = best_param_k_75[prod])
  knn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_knn = knn.predict(X_test.loc[prod])

  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_75)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)



  pscr = prescriptiveness_score(Y_test.loc[prod], preds_knn, preds_saa ,cu = cu, co = co_75)
  result_knn_75_pscr.append(pscr)

In [37]:
KNN_separate_pscr_75 = pd.DataFrame(result_knn_75_pscr, columns=['KNN_pscr_75'])
KNN_separate_pscr_75.index.name = 'item_id'

# knn for 50% service level

## randomized search
*   an empty list is created for every type of hyperparameter
*   in a for-loop the randomized search is executed for every item with cu and co of alpha = 0,5
*   the best hyperparams are calculated and added to the corresponding list

In [ ]:
best_param_k_50 = []
for prod in prods:
  
  
  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_50)
  
  random = RandomizedSearchCV(knn, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
  
  k =  k = random.best_params_.get('n_neighbors')
  best_param_k_50.append(k)

## Newsvendor-Model
*   the data-driven newsvendor model is executed in a loop for every single item (with optimal hyperparameters)
*   the average_costs for every item are calculated (later the prescritiveness score will be used for better comparability)

In [43]:
result_knn_avc_50 = []
for prod in prods:
  
  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_50, n_neighbors = best_param_k_50[prod])
  knn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds = knn.predict(X_test.loc[prod])

  avc = round(average_costs(Y_test.loc[prod], preds, cu = cu, co = co_50),2)
  result_knn_avc_50.append(avc)

In [44]:
KNN_separate_avc_50 = pd.DataFrame(result_knn_avc_50, columns=['KNN_avc_50'])
KNN_separate_avc_50.index.name = 'item_id'

In [45]:
result_knn_50_pscr = []
for prod in prods:
  
  knn = KNeighborsWeightedNewsvendor(cu = cu, co = co_50, n_neighbors = best_param_k_50[prod])
  knn.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_knn = knn.predict(X_test.loc[prod])

  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_50)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)



  pscr = prescriptiveness_score(Y_test.loc[prod], preds_knn, preds_saa ,cu = cu, co = co_50)
  result_knn_50_pscr.append(pscr)

In [46]:
KNN_separate_pscr_50 = pd.DataFrame(result_knn_50_pscr, columns=['KNN_pscr_50'])
KNN_separate_pscr_50.index.name = 'item_id'

# Merging

In [47]:
KNN_separated_avc = pd.concat([KNN_separate_avc_90, KNN_separate_avc_75, KNN_separate_avc_50], axis=1)
KNN_separated_avc

,KNN_avc_90,KNN_avc_75,KNN_avc_50
item_id,,,
0,2.46,5.47,9.46
1,1.00,2.12,3.59
2,2.20,4.67,8.02
3,1.96,4.24,7.56
4,2.30,4.98,9.46
5,1.34,3.04,5.31
6,1.79,3.95,6.91
7,1.62,3.56,6.76
8,1.26,2.72,5.01


In [48]:
KNN_separated_pscr = pd.concat([KNN_separate_pscr_90, KNN_separate_pscr_75, KNN_separate_pscr_50], axis=1)
KNN_separated_pscr

,KNN_pscr_90,KNN_pscr_75,KNN_pscr_50
item_id,,,
0,0.342101,0.269963,0.249334
1,0.441795,0.461729,0.430851
2,0.244910,0.167372,0.134781
3,0.630184,0.583116,0.509393
4,0.287912,0.194532,0.150602
5,0.358213,0.278808,0.172549
6,0.463736,0.443307,0.419718
7,0.405723,0.258365,0.157344
8,0.351101,0.266114,0.219097


# Saving Files

In [49]:
KNN_separated_avc.to_csv('/content/drive/MyDrive/M5/KNN_separated_avc.csv')

KNN_separated_pscr.to_csv('/content/drive/MyDrive/M5/KNN_separated_pscr.csv')